In [ ]:
import sqlite3
import pandas as pd
import os

# Define the SQLite database path
db_path = '../data/raw/UpstoxData.db'

print("Connecting to the SQLite database")
conn = sqlite3.connect(db_path)

# Read the data from the SQLite database into a Pandas DataFrame
print("Reading data from the SQLite database") 
query = "SELECT * FROM candle_data"
combined_df = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()


In [3]:
import polars as pl
import pandas as pd

file_path = '../data/processed/bearish_largecap_with_indicators.pkl'

combined_df_pandas = pd.read_pickle(file_path)
combined_df_pandas
# combined_df = pl.from_pandas(combined_df_pandas)
# start_date = combined_df['date'].min()
# end_date = combined_df['date'].max()
# print(f"\nDate range in the DataFrame: {start_date} to {end_date}")

# combined_df.head(5)

,date,open,high,low,close,volume,symbol,Market Cap,NIFTY 500,SMA,...,HT_DCPHASE,HT_PHASOR_inphase,HT_PHASOR_quadrature,HT_SINE,HT_LEADSINE,HT_TRENDLINE,HT_TRENDMODE,Nifty500_5EMA,Nifty500_15EMA,y
50828,2007-02-13,575.65,592.35,567.00,576.70,774222,ABB,167340.865112,3324.55,588.655,...,NaN,NaN,NaN,NaN,NaN,NaN,0,3391.772348,3408.712097,582.50
50829,2007-02-14,576.55,585.55,569.20,582.50,566484,ABB,167340.865112,3322.75,590.290,...,NaN,NaN,NaN,NaN,NaN,NaN,0,3368.764898,3397.966835,594.95
50830,2007-02-15,593.95,625.35,585.65,594.95,721088,ABB,167340.865112,3403.70,590.250,...,NaN,NaN,NaN,NaN,NaN,NaN,0,3380.409932,3398.683481,612.90
50831,2007-02-19,609.90,622.40,600.50,612.90,1828346,ABB,167340.865112,3414.65,592.905,...,NaN,NaN,NaN,NaN,NaN,NaN,0,3391.823288,3400.679296,597.20
50832,2007-02-20,616.00,621.60,594.40,597.20,893163,ABB,167340.865112,3373.35,593.165,...,NaN,-6.305202,-18.739462,NaN,NaN,NaN,0,3385.665525,3397.263134,602.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5413344,2024-05-13,991.20,997.00,970.60,992.50,1082684,ZYDUSLIFE,125497.503233,20511.00,985.210,...,128.228081,49.849591,-35.880231,0.785554,0.117917,969.537609,1,20558.820776,20656.204959,983.65
5413345,2024-05-14,995.00,1016.65,978.50,983.65,799850,ZYDUSLIFE,125497.503233,20683.80,987.265,...,134.811859,24.630602,-77.673813,0.709425,0.003284,969.504022,1,20600.480517,20659.654339,997.05
5413346,2024-05-15,991.90,1005.00,983.65,997.05,1174315,ZYDUSLIFE,125497.503233,20741.85,991.615,...,161.550055,2.811684,-60.604853,0.316476,-0.446979,969.158723,1,20647.603678,20669.928797,1009.00
5413360,2024-06-04,1049.00,1049.95,933.80,1003.60,3058666,ZYDUSLIFE,125497.503233,20323.85,1045.885,...,263.319142,-28.345218,-20.431209,-0.993210,-0.784569,1037.790670,1,21037.578368,21100.279139,1048.65


In [ ]:
# Example symbol to filter
symbol_to_filter = "TCS"

# Filter the data for the specific symbol
filtered_df = combined_df.filter(pl.col("symbol") == symbol_to_filter)
filtered_df

In [ ]:
filtered_df_pandas = filtered_df.to_pandas()
filtered_df_pandas

# Check Backtests

In [2]:
import pandas as pd
from scipy.stats import trim_mean

def clean_data(df, trim_fraction=0.1):
    # Function to split annual returns into a dictionary
    def split_annual_returns(annual_return_str):
        if isinstance(annual_return_str, str):
            returns = annual_return_str.split(", ")
            return {
                int(year.strip()): float(ret.strip('%'))
                for item in returns if ": " in item
                for year, ret in [item.split(": ")]
            }
        else:
            return {}

    # Expand the annual returns into separate columns
    annual_returns_df = df['Annual_Return'].apply(split_annual_returns).apply(pd.Series)

    # Combine with the original DataFrame
    df = pd.concat([df, annual_returns_df], axis=1).drop(columns=["Annual_Return"])

    # Ensure numeric columns
    numeric_cols = [
        'Sharpe_Ratio', 'SQN', 'VWR', 'Start_Portfolio_Value', 
        'End_Portfolio_Value', 'buy_price_denominator', 
        'backtest_sell_signal_strength', 'backtest_buy_signal_strength', 
        'backtest_yesterday_signal_strength', 'lookback_days_for_signal'
    ] + list(range(2007, 2025))
    
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Define the years of interest
    years = list(range(2007, 2025))

    # Function to apply trimmed mean
    def trimmed_mean(series):
        return trim_mean(series.dropna(), proportiontocut=trim_fraction)

    # Create a dictionary for aggregation functions
    agg_funcs = {
        'Sharpe_Ratio': trimmed_mean,
        'SQN': trimmed_mean,
        'VWR': trimmed_mean,
        'Start_Portfolio_Value': 'sum',
        'End_Portfolio_Value': 'sum',
        'buy_price_denominator': 'mean',
        'backtest_sell_signal_strength': 'mean',
        'backtest_buy_signal_strength': 'mean',
        'backtest_yesterday_signal_strength': 'mean',
        'lookback_days_for_signal': 'mean'
    }

    # Add trimmed mean for each year
    for year in years:
        agg_funcs[year] = trimmed_mean

    # Perform aggregations grouped on columns: 'Strategy', 'Backtest_Time'
    df = df.groupby(['Strategy', 'Backtest_Time']).agg(agg_funcs).reset_index()

    # Rename the aggregated columns to indicate trimmed mean
    df = df.rename(columns={col: f'{col}_trimmed_mean' for col in years})
    df = df.rename(columns={
        'Sharpe_Ratio': 'Sharpe_Ratio_trimmed_mean',
        'SQN': 'SQN_trimmed_mean',
        'VWR': 'VWR_trimmed_mean'
    })

    return df

# Load the data
df = pd.read_csv(r'd:\code\trading_system\data\backtest_results.csv')
# df = pd.read_csv(r'D:\code\trading_system\data\archive\backtest_results_sell_signal_strength_multiple_models.csv')

# Clean the data
df_clean = clean_data(df.copy())

# Display the cleaned data
print(df_clean)


                   Strategy       Backtest_Time  Sharpe_Ratio_trimmed_mean  \
0           multiple_models  05-Aug-24 21:37:16                   0.284215   
1           multiple_models  05-Aug-24 21:37:17                   0.238795   
2  multiple_models_trailing  05-Aug-24 21:33:31                  -0.415703   
3  multiple_models_trailing  05-Aug-24 21:33:32                  -0.799369   

   SQN_trimmed_mean  VWR_trimmed_mean  Start_Portfolio_Value  \
0          0.278179          2.223703               83500000   
1          0.083539          2.253823               27000000   
2          0.557014          0.321347               82000000   
3          0.517239          0.344937               28500000   

   End_Portfolio_Value  buy_price_denominator  backtest_sell_signal_strength  \
0         3.016408e+08                    4.0                           -6.0   
1         1.024674e+08                    4.0                           -6.0   
2         9.227774e+07                    4.0   

In [ ]:
import pandas as pd

df = pd.read_csv(r'..\notebooks\Tickertape - Market Movers - 18-Jul-24.csv')

df

In [1]:
import pandas as pd

# Load the data
file_path = '/mnt/data/Tickertape - Market Movers - 18-Jul-24.csv'
data = pd.read_csv(file_path)

# Fill or drop the null values in 'Transaction Type' column
data = data.dropna(subset=['Transaction Type'])

# Preprocess and aggregate data by stock
grouped = data.groupby('Stocks').agg(
    total_quantity=pd.NamedAgg(column='Quantity', aggfunc='sum'),
    total_value=pd.NamedAgg(column='Value Traded (₹)', aggfunc='sum'),
    avg_holdings_change=pd.NamedAgg(column='Holdings change(%)', aggfunc='mean'),
    buy_count=pd.NamedAgg(column='Transaction Type', aggfunc=lambda x: (x.str.lower() == 'buy').sum()),
    sell_count=pd.NamedAgg(column='Transaction Type', aggfunc=lambda x: (x.str.lower() == 'sell').sum()),
    institutional_weight=pd.NamedAgg(column='Category', aggfunc=lambda x: (x.str.lower().str.contains('institutional')).sum()),
    promoter_weight=pd.NamedAgg(column='Category', aggfunc=lambda x: (x.str.lower().str.contains('promoter')).sum()),
    insider_weight=pd.NamedAgg(column='Category', aggfunc=lambda x: (x.str.lower().str.contains('insider')).sum()),
    hni_weight=pd.NamedAgg(column='Category', aggfunc=lambda x: (x.str.lower().str.contains('hni')).sum())
)

# Define a sophisticated scoring function
def sophisticated_score(row):
    score = 0
    # Score based on total quantity and value
    score += min(row['total_quantity'] / 10000, 5)  # Cap at 5
    score += min(row['total_value'] / 10000000, 5)  # Cap at 5
    
    # Score based on average holdings change
    score += min(row['avg_holdings_change'] * 100, 5)  # Cap at 5
    
    # Score based on transaction counts
    score += row['buy_count'] * 2 - row['sell_count'] * 2  # +2 for buy, -2 for sell
    
    # Score based on category weightage
    score += row['institutional_weight'] * 2
    score += row['promoter_weight'] * 2
    score += row['insider_weight'] * 1
    score += row['hni_weight'] * 1
    
    return score

# Apply the sophisticated scoring function to each stock
grouped['Score'] = grouped.apply(sophisticated_score, axis=1)

# Normalize scores to be between 1 and 10
min_score = grouped['Score'].min()
max_score = grouped['Score'].max()
grouped['Normalized_Score'] = 1 + 9 * (grouped['Score'] - min_score) / (max_score - min_score)

import ace_tools as tools; tools.display_dataframe_to_user(name="Sophisticated Stock Scores", dataframe=grouped)

# Display the results sorted by score
grouped.sort_values(by='Normalized_Score', ascending=False, inplace=True)
grouped.head(10)


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/Tickertape - Market Movers - 18-Jul-24.csv'

# Fetch data func

In [1]:
import requests
import pandas as pd
import gzip
import json
from io import BytesIO
import sqlite3
import os
import datetime


# Configuration
url = 'https://assets.upstox.com/market-quote/instruments/exchange/NSE.json.gz'
db_path = 'D:/code/trading_system/data/raw/UpstoxData.db'
start_date = '2007-01-01'
end_date = datetime.datetime.now().strftime('%Y-%m-%d')

# Fetch data from the API
response = requests.get(url)
response.raise_for_status()
with gzip.GzipFile(fileobj=BytesIO(response.content)) as f:
    json_bytes = f.read()
json_str = json_bytes.decode('utf-8')
data = json.loads(json_str)

# Convert data to DataFrame and filter
df = pd.DataFrame(data)
df


,weekly,segment,name,exchange,expiry,instrument_type,asset_symbol,underlying_symbol,instrument_key,lot_size,...,asset_key,underlying_key,tick_size,asset_type,underlying_type,trading_symbol,strike_price,isin,short_name,security_type
0,False,NSE_FO,ADITYA BIRLA FASHION & RT,NSE,1.724956e+12,PE,ABFRL,ABFRL,NSE_FO|49902,2600.0,...,NSE_EQ|INE647O01011,NSE_EQ|INE647O01011,5.00,EQUITY,EQUITY,ABFRL 355 PE 29 AUG 24,355.00,NaN,NaN,NaN
1,NaN,NSE_EQ,GTPL HATHWAY LIMITED,NSE,NaN,EQ,NaN,NaN,NSE_EQ|INE869I01013,1.0,...,NaN,NaN,1.00,NaN,NaN,GTPL,NaN,INE869I01013,GTPL Hathway,NORMAL
2,False,NCD_FO,JPYINR,NSE,1.730227e+12,CE,JPYINR,JPYINR,NCD_FO|14260,1.0,...,NaN,NaN,0.25,CUR,CUR,JPYINR 54.75 CE 29 OCT 24,54.75,NaN,NaN,NaN
3,False,NCD_FO,JPYINR,NSE,1.730227e+12,PE,JPYINR,JPYINR,NCD_FO|14261,1.0,...,NaN,NaN,0.25,CUR,CUR,JPYINR 54.75 PE 29 OCT 24,54.75,NaN,NaN,NaN
4,NaN,NSE_EQ,PRITIKA ENG COMPO LTD,NSE,NaN,SM,NaN,NaN,NSE_EQ|INE0MJQ01012,2000.0,...,NaN,NaN,5.00,NaN,NaN,PRITIKA,NaN,INE0MJQ01012,NaN,SME
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46808,False,NCD_FO,EURINR,NSE,1.735324e+12,CE,EURINR,EURINR,NCD_FO|1763,1.0,...,NaN,NaN,0.25,CUR,CUR,EURINR 91.5 CE 27 DEC 24,91.50,NaN,NaN,NaN
46809,True,NCD_FO,JPYINR,NSE,1.724438e+12,PE,JPYINR,JPYINR,NCD_FO|1765,1.0,...,NaN,NaN,0.25,CUR,CUR,JPYINR 61.75 PE 23 AUG 24,61.75,NaN,NaN,NaN
46810,False,NCD_FO,EURINR,NSE,1.735324e+12,PE,EURINR,EURINR,NCD_FO|1768,1.0,...,NaN,NaN,0.25,CUR,CUR,EURINR 91.5 PE 27 DEC 24,91.50,NaN,NaN,NaN
46811,False,NCD_FO,EURINR,NSE,1.750963e+12,FUT,EURINR,EURINR,NCD_FO|1767,1.0,...,NaN,NaN,0.25,CUR,CUR,EURINR FUT 26 JUN 25,0.00,NaN,NaN,NaN


In [2]:

df_filtered = df[(df['instrument_type'].isin(["EQ", "BE", "SM"])) & 
                    (~df['isin'].astype(str).str.startswith("INF") | df['isin'].isna())]
trading_symbols = df_filtered['trading_symbol'].unique().tolist()
trading_symbols

['GTPL',
 'PRITIKA',
 'SUPRAJIT',
 'KCK',
 'BATAINDIA',
 'MARUTI',
 'BYKE',
 'NATIONALUM',
 'TRIDENT',
 'MBLINFRA',
 'TRIL',
 'NDL',
 'RADIOCITY',
 'GOKULAGRO',
 'TECHNOE',
 'TRIVENI',
 'STARTECK',
 'NINSYS',
 'LEMONTREE',
 'WESTLIFE',
 'BEDMUTHA',
 'LAURUSLABS',
 'VLEGOV',
 'ENDURANCE',
 'MOIL',
 'BAJAJFINSV',
 'FMNL',
 'BCONCEPTS',
 'SHANTI',
 'MOS',
 'AMBANIORGO',
 'JIOFIN',
 'MANAPPURAM',
 'DRREDDY',
 'JOCIL',
 'MEDIASSIST',
 'SMLT',
 'VINYLINDIA',
 'SEJALLTD',
 'IOLCP',
 'ROLLT',
 'MIRZAINT',
 'ZENTEC',
 'CENTRUM',
 'PENINLAND',
 'ASPINWALL',
 'SONAMAC',
 'SHAREINDIA',
 'BOMDYEING',
 'VERTOZ',
 'SGL',
 'INGERRAND',
 'PVSL',
 'JISLJALEQS',
 'GOYALALUM',
 'POLYPLEX',
 'KAMDHENU',
 'STARHEALTH',
 'ESTER',
 'LYPSAGEMS',
 'BANSWRAS',
 'HINDCON',
 'UNICHEMLAB',
 'OWAIS',
 'FIEMIND',
 'SFL',
 'PLATIND',
 'MAHLOG',
 'PREMEXPLN',
 'ZOTA',
 'AMBIKCO',
 'MUNJALAU',
 'BORORENEW',
 'DMCC',
 'GRINFRA',
 'TATVA',
 'CHALET',
 'SAKAR',
 'INDOWIND',
 'DALBHARAT',
 'KHANDSE',
 'SIGNPOST',
 'HTMEDIA'

In [ ]:

# Create SQLite database connection
os.makedirs(os.path.dirname(db_path), exist_ok=True)
if os.path.exists(db_path):
    os.remove(db_path)
db_conn = sqlite3.connect(db_path)

# Create new table
db_conn.execute('''
CREATE TABLE candle_data (
    date TEXT,
    open REAL,
    high REAL,
    low REAL,
    close REAL,
    volume INTEGER,
    symbol TEXT
)
''')

# Fetch and save data for each trading symbol
for symbol in trading_symbols:
    instrument_key = df[df['trading_symbol'] == symbol]['instrument_key'].values[0]
    if instrument_key:
        candle_url = f'https://api.upstox.com/v2/historical-candle/{instrument_key}/day/{end_date}/{start_date}'
        try:
            response = requests.get(candle_url, headers={'Accept': 'application/json'})
            response.raise_for_status()
            candle_data = response.json()
            candles = candle_data['data']['candles']
            candle_df = pd.DataFrame(candles, columns=['date', 'open', 'high', 'low', 'close', 'volume', 'unknown'])
            candle_df.drop(columns=['unknown'], inplace=True)
            candle_df['symbol'] = symbol
            candle_df.to_sql('candle_data', db_conn, if_exists='append', index=False)
        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")
    else:
        print(f"Instrument key for {symbol} not found.")

# Close the database connection
db_conn.close()
print("Data saved to SQLite database")



In [ ]:

# Convert data to DataFrame and filter
df = pd.DataFrame(data)

df_filtered = df[(df['instrument_type'].isin(["EQ", "BE", "SM"])) & 
                    (~df['isin'].astype(str).str.startswith("INF") | df['isin'].isna())]
trading_symbols = df_filtered['trading_symbol'].unique().tolist()

# Create SQLite database connection
os.makedirs(os.path.dirname(db_path), exist_ok=True)
if os.path.exists(db_path):
    os.remove(db_path)
db_conn = sqlite3.connect(db_path)

# Create new table
db_conn.execute('''
CREATE TABLE candle_data (
    date TEXT,
    open REAL,
    high REAL,
    low REAL,
    close REAL,
    volume INTEGER,
    symbol TEXT
)
''')

# Fetch and save data for each trading symbol
for symbol in trading_symbols:
    instrument_key = df[df['trading_symbol'] == symbol]['instrument_key'].values[0]
    if instrument_key:
        candle_url = f'https://api.upstox.com/v2/historical-candle/{instrument_key}/day/{end_date}/{start_date}'
        try:
            response = requests.get(candle_url, headers={'Accept': 'application/json'})
            response.raise_for_status()
            candle_data = response.json()
            candles = candle_data['data']['candles']
            candle_df = pd.DataFrame(candles, columns=['date', 'open', 'high', 'low', 'close', 'volume', 'unknown'])
            candle_df.drop(columns=['unknown'], inplace=True)
            candle_df['symbol'] = symbol
            candle_df.to_sql('candle_data', db_conn, if_exists='append', index=False)
        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")
    else:
        print(f"Instrument key for {symbol} not found.")

# Close the database connection
db_conn.close()
print("Data saved to SQLite database")



# Market cap

In [3]:
import pandas as pd
market_cap_csv_path = '../data/tickertape/mkt_cap.csv'
market_cap_df = pd.read_csv(market_cap_csv_path)
market_cap_df

,Name,Ticker,Sub-Sector,Market Cap
0,Reliance Industries Ltd,RELIANCE,Oil & Gas - Refining & Marketing,2.037084e+06
1,Tata Consultancy Services Ltd,TCS,IT Services & Consulting,1.586658e+06
2,HDFC Bank Ltd,HDFCBANK,Private Banks,1.230795e+06
3,Bharti Airtel Ltd,BHARTIARTL,Telecom Services,8.911909e+05
4,ICICI Bank Ltd,ICICIBANK,Private Banks,8.551637e+05
...,...,...,...,...
5001,Chetana Education Ltd,CHETANA,NaN,NaN
5002,Manglam Infra & Engineering Ltd,MIEL,NaN,NaN
5003,Trom Industries Ltd,TROM,NaN,NaN
5004,Clinitech Laboratory Ltd,CTLLAB,NaN,NaN
